## Use Keras Models with Scikit-Learn 

- Cross Vaildation
- Tune parameters of model

In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"

### Cross-Validation

- Use KerasClassifier and KerasRegressor classes in Keras
- Use StratifiedKFold to perform N-fold stratified cross-validation

In [10]:
# MLP for Pima Indians Dataset with N-fold cross validation via sklearn

# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

seed = 7
np.random.seed(7)
dataset = pd.read_csv('diabetes.csv')
dataset.head()

#split into input and output
X = np.array(dataset.iloc[:, 0:8])
Y = np.array(dataset.iloc[:, 8])

# create model
model = KerasClassifier(build_fn=create_model, epochs=150, batch_size=100,verbose=0)
# evaluate using N-fold cross validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)
results = cross_val_score(model,X,Y,cv=kfold)
print("The test results are {}".format(results.mean()))

The test results are 0.6900196489991095


### Grid Search

Use GridSearchCV class to determine:
- Optimizers for searching different weight values.
- Initializers for preparing the network weights using different schemes.
- Number of epochs for training the model for different number of exposures to the training dataset.
- Batches for varying the number of samples before weight updates

Should be used only lightly as models take a lot of computation

In [16]:
# Function to create model, required for KerasClassifier
def create_model(optimizer='rmsprop', init='glorot_uniform'):
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, kernel_initializer=init, activation='relu'))
    model.add(Dense(8, kernel_initializer=init, activation='relu'))
    model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

seed = 7
np.random.seed(7)
dataset = pd.read_csv('diabetes.csv')
dataset.head()

#split into input and output
X = np.array(dataset.iloc[:, 0:8])
Y = np.array(dataset.iloc[:, 8])

#create the model
model = KerasClassifier(build_fn=create_model, verbose = 0)

#grid search epochs, batch size, optimizer
optimizers = ['rmsprop','adam']
inits = ['glorot_uniform','normal','uniform']
epochs = [150]
batches = [100]
param_grid = dict(optimizer = optimizers, epochs=epochs, batch_size=batches, init=inits)
grid = GridSearchCV(estimator=model,param_grid=param_grid)
grid_result = grid.fit(X,Y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Best: 0.699219 using {'batch_size': 100, 'epochs': 150, 'init': 'uniform', 'optimizer': 'adam'}
0.673177 (0.013279) with: {'batch_size': 100, 'epochs': 150, 'init': 'glorot_uniform', 'optimizer': 'rmsprop'}
0.674479 (0.012075) with: {'batch_size': 100, 'epochs': 150, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.697917 (0.018688) with: {'batch_size': 100, 'epochs': 150, 'init': 'normal', 'optimizer': 'rmsprop'}
0.677083 (0.014382) with: {'batch_size': 100, 'epochs': 150, 'init': 'normal', 'optimizer': 'adam'}
0.696615 (0.016053) with: {'batch_size': 100, 'epochs': 150, 'init': 'uniform', 'optimizer': 'rmsprop'}
0.699219 (0.022999) with: {'batch_size': 100, 'epochs': 150, 'init': 'uniform', 'optimizer': 'adam'}
